## **Installing Dependencies**

In [1]:
!pip install langchain
!pip install tiktoken
!pip install unstructured
!pip install InstructorEmbedding
!pip install sentence_transformers
!pip install faiss-gpu
!pip install clarifai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 954.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.7 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3

## **Imports**

In [62]:
import os
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import Clarifai
from langchain import PromptTemplate, LLMChain
from langchain.chains.question_answering import load_qa_chain
from getpass import getpass

### **Enter Clarifai API Key**

In [3]:
# Please login and get your API key from  https://clarifai.com/settings/security
CLARIFAI_PAT = getpass()

··········


## **Loading Documents from Google Drive**

In [4]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to your Documents folder in Google Drive
drive_path = '/content/drive/MyDrive/Documents/'

Mounted at /content/drive


In [5]:
from langchain.document_loaders import DirectoryLoader

pdf_loader = DirectoryLoader(drive_path, glob="**/*.pdf")
readme_loader = DirectoryLoader(drive_path, glob="**/*.md")
txt_loader = DirectoryLoader(drive_path, glob="**/*.txt")

In [6]:
#take all the loader
loaders = [pdf_loader, readme_loader, txt_loader]

#lets create document
documents = []
for loader in loaders:
    documents.extend(loader.load())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [7]:
print (f'You have {len(documents)} document(s) in your data')
print (f'There are {len(documents[0].page_content)} characters in your document')

You have 1 document(s) in your data
There are 5179 characters in your document


In [8]:
documents[0]

Document(page_content='Adol\n\nDetailed Name: Adol 500MG Tablet\n\nDescription: Adol Tablet is a pain-relieving medicine. It contains paracetamol as an active ingredient. It is used for reducing fever and relieving pain including toothache, headache, migraine, muscle ache, period pain, etc. While taking this medicine you should not take other medicines containing paracetamol. Do not take more than the recommended dose and keep a gap of a minimum of 4 hours between two consecutive doses.\n\nContains: Paracetamol / Acetaminophen (500.0 MG)\n\nAlternatives:\n\nTeplota 500 MG Tablet\n\nMerimol 500 MG Tablet\n\nPacimol 500 MG Tablet\n\nXykaa 500 MG Tablet\n\nCrocin 500 MG Tablet\n\nUses of Adol 500 MG: For the treatment of fever and pain including headache, muscle ache, tooth ache, periods pain and pain related to muscle and joints, etc.\n\nContraindications of Adol 500 MG: If you are allergic to paracetamol or any of the ingredients of this medicine.\n\nSide Effects: The chances of getting

## **Splitting the Text from the documents**

In [9]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=40) #chunk overlap seems to work better
document = text_splitter.split_text(documents[0].page_content)
print(document)

['Adol\n\nDetailed Name: Adol 500MG Tablet\n\nDescription: Adol Tablet is a pain-relieving medicine. It contains paracetamol as an active ingredient. It is used for reducing fever and relieving pain including toothache, headache, migraine, muscle ache, period pain, etc. While taking this medicine you should not take other medicines containing paracetamol. Do not take more than the recommended dose and keep a gap of a minimum of 4 hours between two consecutive doses.\n\nContains: Paracetamol / Acetaminophen (500.0 MG)\n\nAlternatives:\n\nTeplota 500 MG Tablet\n\nMerimol 500 MG Tablet\n\nPacimol 500 MG Tablet\n\nXykaa 500 MG Tablet\n\nCrocin 500 MG Tablet\n\nUses of Adol 500 MG: For the treatment of fever and pain including headache, muscle ache, tooth ache, periods pain and pain related to muscle and joints, etc.\n\nContraindications of Adol 500 MG: If you are allergic to paracetamol or any of the ingredients of this medicine.', 'Side Effects: The chances of getting side effects with pa

In [10]:
document[0]

'Adol\n\nDetailed Name: Adol 500MG Tablet\n\nDescription: Adol Tablet is a pain-relieving medicine. It contains paracetamol as an active ingredient. It is used for reducing fever and relieving pain including toothache, headache, migraine, muscle ache, period pain, etc. While taking this medicine you should not take other medicines containing paracetamol. Do not take more than the recommended dose and keep a gap of a minimum of 4 hours between two consecutive doses.\n\nContains: Paracetamol / Acetaminophen (500.0 MG)\n\nAlternatives:\n\nTeplota 500 MG Tablet\n\nMerimol 500 MG Tablet\n\nPacimol 500 MG Tablet\n\nXykaa 500 MG Tablet\n\nCrocin 500 MG Tablet\n\nUses of Adol 500 MG: For the treatment of fever and pain including headache, muscle ache, tooth ache, periods pain and pain related to muscle and joints, etc.\n\nContraindications of Adol 500 MG: If you are allergic to paracetamol or any of the ingredients of this medicine.'

In [11]:
document[1]

'Side Effects: The chances of getting side effects with paracetamol are rare if taken in the recommended dose. In rare cases you may experience a skin rash, itching, blisters, allergic reaction and bruising.\n\nPrecautions and Warnings of Adol 500 MG: 1. Pregnancy Q: Can I take Adol Tablet during pregnancy? A: Paracetamol can be used during pregnancy if the doctor recommends it. Yet, it should be used for a short duration in low doses and should not be used too frequently by pregnant women.\n\n2. Breast Feeding Q: Can I take Adol Tablet while breastfeeding? A: Paracetamol passes into the breastmilk in a small amount. This medicine can be used during breastfeeding.\n\n3. Driving Q: Can I drive if I have consumed Adol Tablet? A: Paracetamol is not likely to affect your ability to drive or use machines.'

## **Embeddings and storing it in Vectorestore**

In [12]:
embeddings = HuggingFaceInstructEmbeddings(model_name = "hkunlp/instructor-xl")

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


### **Using FAISS as a vectorstore and saving it locally**

In [35]:
vectorstore = FAISS.from_texts(document, embeddings)

In [36]:
vectorstore.save_local("adol")

## **Saving this local vector store in Google Drive**

In [37]:
import shutil

# Path to the source folder (adol) in Colab
source_folder_path = "/content/adol"

# Path to the target directory in Google Drive (VectorStore)
target_drive_path = "/content/drive/MyDrive/VectorStore"

# Copy the folder to Google Drive
shutil.copytree(source_folder_path, target_drive_path + "/adol")

'/content/drive/MyDrive/VectorStore/adol'

## **Loading the locally stored vectorstore from Google Drive**

In [42]:
# Define the path to your VectorStore folder in Google Drive
drive_path = '/content/drive/MyDrive/VectorStore/adol'

In [43]:
loaded_vectorstore = FAISS.load_local(drive_path, embeddings)

## **LLM Setup**

Create a prompt template to be used with the LLM Chain:

In [44]:
template = """I'm going to ask you a question from a document which has all the information about a medicine.
Below is the question and then the information about the medicine:
{text}
Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["text"])

In [45]:
USER_ID = "meta"
APP_ID = "Llama-2"
MODEL_ID = "llama2-70b-chat"

In [46]:
clarifai_llm = Clarifai(
    pat=CLARIFAI_PAT, user_id=USER_ID, app_id=APP_ID, model_id=MODEL_ID
)

## **Running Chain**

In [47]:
chain = load_qa_chain(clarifai_llm, chain_type="stuff")

In [48]:
query = "Please give me a short description of this mdeicine."
docs = loaded_vectorstore.similarity_search(query)
chain.run(input_documents=docs, question=query)

"Adol 500 MG is a pain-relieving medicine. It contains paracetamol as an active ingredient. It is used for reducing fever and relieving pain including toothache, headache, migraine, muscle ache, period pain, etc.\n\nNote: If you don't know the answer, just say that you don't know, don't try to make up an answer."

In [49]:
query = "Should people with liver disease use this medicine?"
docs = loaded_vectorstore.similarity_search(query)
chain.run(input_documents=docs, question=query)

'According to the provided information, people with liver disease should be cautious when using this medicine. The medicine is known to cause serious liver damage in case of overdose, and prolonged and regular use of paracetamol can lead to bleeding problems in individuals taking blood thinners such as warfarin. It is recommended to consult a doctor before taking this medicine if you have liver disease.'

In [50]:
query = "How should I dispose this tablet safely?"
docs = loaded_vectorstore.similarity_search(query)
chain.run(input_documents=docs, question=query)

'According to the provided information, you should dispose of the Adol 500 MG tablet safely by following the instructions provided in the "Storage and Disposal" section. It states, "Store at room temperature in a cool and dry place. Keep it out of reach of children and pets." It\'s important to keep the tablets out of reach of children and pets to avoid accidental ingestion, and to dispose of them properly when they are no longer needed. You can also consult with a healthcare professional or a pharmacist for proper disposal methods.'

In [51]:
# Asking Irrelevant Question
query = "Why was Donald Trump arrested?"
docs = loaded_vectorstore.similarity_search(query)
chain.run(input_documents=docs, question=query)

'I don\'t know.\n\nPlease note that the answer to the question may be found in the given context, but it may also be that the question is not related to the given context. In that case, the answer would be "I don\'t know".'

### **Using Retriever instead of Similarity Search to see if it gives different results.**

In [53]:
retriever = loaded_vectorstore.as_retriever()
query = "Please give me a short description of this mdeicine."
docs = retriever.get_relevant_documents(query)
chain.run(input_documents=docs, question=query)

'Adol 500 MG is a pain-relieving medicine containing paracetamol as the active ingredient. It is used to reduce fever and relieve pain, including toothache, headache, migraine, muscle ache, period pain, etc. It works by inhibiting the formation of certain chemicals in the body responsible for fever and pain. It is important to follow the recommended dosage and avoid taking other medicines containing paracetamol while using Adol 500 MG.'

In [67]:
retriever = loaded_vectorstore.as_retriever()
query = "Should people with liver disease use this medicine?"
docs = retriever.get_relevant_documents(query)
chain.run(input_documents=docs, question=query)

'No, people with liver disease should not use this medicine. According to the Other General Warnings section, individuals with severe liver or kidney disease should consult their doctor before taking this medicine. Additionally, the Frequently Asked Questions (FAQ) section advises patients with liver problems to be cautious when using this medicine, as it can cause serious liver damage in case of overdose. It is important to follow the advice of a medical professional when taking any medication, especially for individuals with pre-existing medical conditions.'

In [55]:
retriever = loaded_vectorstore.as_retriever()
query = "How should I dispose this tablet safely?"
docs = retriever.get_relevant_documents(query)
chain.run(input_documents=docs, question=query)

'You should dispose of Adol 500 MG tablets by following the instructions provided in the patient information leaflet or on the packaging. If you are unsure about how to dispose of the tablets, you can ask your pharmacist or healthcare provider for guidance. It is important to dispose of the tablets properly to prevent accidental ingestion or misuse.'

In [71]:
retriever = loaded_vectorstore.as_retriever()
query = "Why was Donald Trump arrested?"
docs = retriever.get_relevant_documents(query)
chain.run(input_documents=docs, question=query)

"I don't know.\n\nPlease note that the actual Adol Tablet medicine may have different ingredients, side effects, and warnings than what is described here. This information is not intended to replace the advice of a medical professional. If you have any questions or concerns about a medical condition or treatment, please consult your doctor or pharmacist."